In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import random

# Configuración del dispositivo (GPU si está disponible, vital para Senior devs)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

class TextProcessor:
    def __init__(self, text_data):
        # Crear vocabulario de caracteres únicos
        chars = sorted(list(set(text_data)))
        self.vocab_size = len(chars) # +1 para el token EOS (End of Sequence) si es necesario, o manejamos \n
        self.char_to_ix = {ch: i for i, ch in enumerate(chars)}
        self.ix_to_char = {i: ch for i, ch in enumerate(chars)}
        self.chars = chars

    def str_to_tensor(self, seq):
        """Convierte string a tensor de índices (LongTensor)"""
        tensor = torch.tensor([self.char_to_ix[ch] for ch in seq], dtype=torch.long).to(device)
        return tensor

# Datos dummy para probar el código inmediatamente
data_raw = "diplosaurio\ntiranosaurio\nvelociraptor\ntriceratops\nestegosaurio\n"
processor = TextProcessor(data_raw)
processor

cuda


In [6]:
class DinoRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size, num_layers=1):
        super(DinoRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # 1. Capa de Embedding: Representación densa de x_t
        # Equivalente eficiente a la entrada one-hot
        self.embedding = nn.Embedding(vocab_size, hidden_size)

        # 2. Capa RNN: Procesa la secuencia
        # batch_first=True espera entrada (batch, seq, features)
        self.rnn = nn.RNN(input_size=hidden_size,
                          hidden_size=hidden_size,
                          num_layers=num_layers,
                          batch_first=True)

        # 3. Capa de Salida (Decoder): W_ya * a_t + b_y
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, hidden):
        # x shape: (batch_size, seq_len)
        # hidden shape: (num_layers, batch_size, hidden_size)

        # Embed: (batch, seq, hidden)
        x = self.embedding(x)

        # RNN Forward: out contiene los estados a_t para cada paso
        out, hidden = self.rnn(x, hidden)

        # Flatten output para la capa lineal
        out = out.reshape(-1, self.hidden_size)

        # Predicción (Logits no normalizados)
        out = self.fc(out)

        return out, hidden

    def init_hidden(self, batch_size):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)

In [7]:
def train_step(model, optimizer, criterion, input_seq, target_seq):
    model.train() # Modo entrenamiento

    # Inicializar estado oculto (a_0 = 0)
    hidden = model.init_hidden(batch_size=1)

    model.zero_grad()
    loss = 0

    # PyTorch permite pasar toda la secuencia a la vez en nn.RNN
    # input_seq: (1, seq_len)
    input_tensor = input_seq.unsqueeze(0)
    target_tensor = target_seq.view(-1)   # Flatten targets

    output, _ = model(input_tensor, hidden)

    # output: (seq_len, vocab_size), target: (seq_len)
    loss = criterion(output, target_tensor)

    loss.backward()

    # Gradient Clipping: CRUCIAL en RNNs para evitar explosión de gradiente
    nn.utils.clip_grad_norm_(model.parameters(), 5)

    optimizer.step()

    return loss.item()

# Configuración
hidden_size = 128
model = DinoRNN(processor.vocab_size, hidden_size).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
criterion = nn.CrossEntropyLoss()

# Ejemplo de una iteración (en la práctica, iterarías sobre tu dataset)
input_str = "tiranosaurio"
input_tensor = processor.str_to_tensor(input_str[:-1]) # entrada: "tiranosauri"
target_tensor = processor.str_to_tensor(input_str[1:]) # target:  "iranosaurio"

loss = train_step(model, optimizer, criterion, input_tensor, target_tensor)
print(f"Loss inicial: {loss:.4f}")

Loss inicial: 2.7733


In [8]:
def sample(model, start_char='t', max_length=20, temperature=1.0):
    model.eval() # Modo evaluación

    with torch.no_grad():
        # Inicializar entrada
        input_tensor = processor.str_to_tensor(start_char).unsqueeze(0) # (1, 1)
        hidden = model.init_hidden(1)

        generated_name = start_char

        for _ in range(max_length):
            output, hidden = model(input_tensor, hidden)

            # Aplicar temperatura para controlar la aleatoriedad
            # Temp alta (>1): más creativo/errores. Temp baja (<1): conservador/repetitivo.
            output_dist = output.data.view(-1).div(temperature).exp()
            top_i = torch.multinomial(output_dist, 1)[0]

            predicted_char = processor.ix_to_char[top_i.item()]

            # Si el modelo predice el fin de línea, terminamos
            if predicted_char == '\n':
                break

            generated_name += predicted_char
            input_tensor = torch.tensor([[top_i]], dtype=torch.long).to(device)

        return generated_name

# Prueba de generación (sin entrenar saldrá basura, pero el código es funcional)
print(f"Generado: {sample(model, start_char='a', temperature=0.8)}")

Generado: aug


In [ ]:
import torch
import torch.nn as nn

# --- 1. Definición de Hiperparámetros ---
BATCH_SIZE = 2      # Número de muestras procesadas en paralelo
SEQ_LEN = 5         # Longitud de la secuencia temporal (T)
INPUT_SIZE = 3      # Número de características por paso (ej. [temp, presión, humedad])
HIDDEN_SIZE = 4     # Dimensión del espacio latente (memoria de la red)

# --- 2. Instancia de la Capa RNN ---
# batch_first=True es crucial para usar el formato (N, L, H_in) estándar en ingeniería
rnn_layer = nn.RNN(input_size=INPUT_SIZE,
                   hidden_size=HIDDEN_SIZE,
                   num_layers=1,
                   batch_first=True)

# --- 3. Generación de Entrada Sintética ---
# Tensor X: [Batch Size, Sequence Length, Input Features]
inputs = torch.randn(BATCH_SIZE, SEQ_LEN, INPUT_SIZE)

print(f"Dimensiones de Entrada (X): {inputs.shape}")
print("-" * 30)

# --- 4. Inicialización del Estado Oculto (Opcional) ---
# Si no se provee, PyTorch asume ceros.
# Formato h0: [Num Layers, Batch Size, Hidden Size]
h0 = torch.zeros(1, BATCH_SIZE, HIDDEN_SIZE)

# --- 5. Forward Pass ---
# La RNN devuelve dos objetos:
#   output: contiene todos los estados ocultos h_1, h_2, ..., h_T
#   h_n: contiene solo el último estado oculto h_T (memoria final)
output, h_n = rnn_layer(inputs, h0)

# --- 6. Análisis de Salidas ---
print(f"Dimensiones de 'output' (Secuencia completa): {output.shape}")
print(f"Dimensiones de 'h_n'    (Estado final):       {h_n.shape}")

print("\n--- Verificación de Valores ---")
print("El último paso de tiempo de 'output' debe ser igual a 'h_n' (para una sola capa):")
# Comparamos el último paso temporal de la primera muestra en el batch
print(f"Output (t=5): {output[0, -1, :].detach().numpy()}")
print(f"Hidden (Final): {h_n[0, 0, :].detach().numpy()}")

Dimensiones de Entrada (X): torch.Size([2, 5, 3])
------------------------------
Dimensiones de 'output' (Secuencia completa): torch.Size([2, 5, 4])
Dimensiones de 'h_n'    (Estado final):       torch.Size([1, 2, 4])

--- Verificación de Valores ---
El último paso de tiempo de 'output' debe ser igual a 'h_n' (para una sola capa):
Output (t=5): [-0.116896   0.3243003 -0.594122  -0.5339937]
Hidden (Final): [-0.116896   0.3243003 -0.594122  -0.5339937]
